# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [4]:
from sklearn.preprocessing import LabelEncoder
encoded = pd.get_dummies(df, columns=["homeowner"])
encoded.head()

# Create our features
X = encoded.drop(columns="loan_status")

# Create our target
y = pd.DataFrame(encoded["loan_status"])

In [5]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [6]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [7]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [8]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [9]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)

In [10]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

# Create the model
lr_model = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model to the data
lr_model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [12]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

In [13]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]])

In [14]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [16]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

# View the count of target classes with Counter
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)


print(Counter(y_train["loan_status"]))
Counter(y_resampled["loan_status"])

Counter({'low_risk': 56277, 'high_risk': 1875})


Counter({'low_risk': 56277, 'high_risk': 56277})

In [17]:
# Train the Logistic Regression model using the resampled data
lr_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = lr_model.predict(X_test_scaled)
bal_a = balanced_accuracy_score(y_test, y_pred)

print(bal_a)

0.9946414201183431


In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, y_pred)
cf_matrix

array([[  622,     3],
       [  111, 18648]])

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
cl_report = classification_report_imbalanced(y_test, y_pred, digits=4)
print(cl_report)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8486    0.9952    0.9941    0.9161    0.9946    0.9894       625
   low_risk     0.9998    0.9941    0.9952    0.9970    0.9946    0.9892     18759

avg / total     0.9950    0.9941    0.9952    0.9943    0.9946    0.9892     19384



### SMOTE Oversampling

In [21]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(X_train, y_train)

# View the count of target classes with Counter
print(Counter(y_train["loan_status"]))
Counter(y_resampled["loan_status"])# YOUR CODE HERE

Counter({'low_risk': 56277, 'high_risk': 1875})


Counter({'low_risk': 56277, 'high_risk': 56277})

In [22]:
# Train the Logistic Regression model using the resampled data
lr_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [23]:
# Calculated the balanced accuracy score
ste_y_pred = lr_model.predict(X_test)
ste_bal_acc = balanced_accuracy_score(y_test, ste_y_pred)
print(ste_bal_acc)

0.9948279972279972


In [24]:
# Display the confusion matrix
ste_cf_matrix = confusion_matrix(y_test, ste_y_pred)
ste_cf_matrix

array([[  622,     3],
       [  104, 18655]])

In [25]:
# Print the imbalanced classification report
ste_cl_report = classification_report_imbalanced(y_test, ste_y_pred, digits=5)
print(ste_cl_report)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk    0.85675   0.99520   0.99446   0.92080   0.99483   0.98976       625
   low_risk    0.99984   0.99446   0.99520   0.99714   0.99483   0.98961     18759

avg / total    0.99523   0.99448   0.99518   0.99468   0.99483   0.98961     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [26]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
print(Counter(y_train["loan_status"]))
print(Counter(y_resampled["loan_status"]))

KeyboardInterrupt: 

In [27]:
# Train the Logistic Regression model using the resampled data
lr_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [30]:
# Calculate the balanced accuracy score
cc_y_pred = lr_model.predict(X_test)
cc_bal_acc = balanced_accuracy_score(y_test, cc_y_pred)
print(cc_bal_acc)

0.9948279972279972


In [31]:
# Display the confusion matrix
cc_cf_matrix = confusion_matrix(y_test, cc_y_pred)
cc_cf_matrix

array([[  622,     3],
       [  104, 18655]])

In [32]:
# Print the imbalanced classification report
cc_cl_report = classification_report_imbalanced(y_test, cc_y_pred, digits=4)
print(cc_cl_report)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8567    0.9952    0.9945    0.9208    0.9948    0.9898       625
   low_risk     0.9998    0.9945    0.9952    0.9971    0.9948    0.9896     18759

avg / total     0.9952    0.9945    0.9952    0.9947    0.9948    0.9896     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [33]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)

# View the count of target classes with Counter
print(Counter(y_train["loan_status"]))
Counter(y_resampled["loan_status"])

Counter({'low_risk': 56277, 'high_risk': 1875})


Counter({'high_risk': 55513, 'low_risk': 55928})

In [34]:
# Train the Logistic Regression model using the resampled data
lr_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [35]:
# Calculate the balanced accuracy score
sten_y_pred = lr_model.predict(X_test)
sten_bal_acc = balanced_accuracy_score(y_test, sten_y_pred)

print(sten_bal_acc)

0.994748035609574


In [36]:
# Display the confusion matrix
sten_cf_matrix = confusion_matrix(y_test, sten_y_pred)
sten_cf_matrix

array([[  622,     3],
       [  107, 18652]])

In [37]:
# Print the imbalanced classification report
sten_cl_report = classification_report_imbalanced(y_test, sten_y_pred, digits=5)
print(sten_cl_report)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk    0.85322   0.99520   0.99430   0.91876   0.99475   0.98961       625
   low_risk    0.99984   0.99430   0.99520   0.99706   0.99475   0.98943     18759

avg / total    0.99511   0.99433   0.99517   0.99454   0.99475   0.98944     19384



# Final Questions

1. Which model had the best balanced accuracy score?

Best balanced accuracy was SMOTE Oversampling Model

2. Which model had the best recall score?

Best recall score is the SMOTE Oversampling model 

3. Which model had the best geometric mean score?

Again it was SMOTE Oversampling model 
